In [128]:
from sage.all import *
import hashlib
import os
import numpy as np
import random
import logging
from typing import Tuple, Optional, List
import time

In [129]:
file_handler = logging.FileHandler('bike_debug.log')
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
logger = logging.getLogger("BIKE_DEBUG")
logger.setLevel(logging.DEBUG)
logger.handlers = [file_handler]  # Substituir handlers existentes
logger.propagate = False  # Evitar propagação para o logger raiz

In [ ]:
# ============================================================================
# PARÂMETROS DE SEGURANÇA (Configuráveis)
# ============================================================================
class SecurityParams:
    """Parâmetros de segurança configuráveis para diferentes níveis NIST"""
    
    def __init__(self, level: int = 1):
        if level == 1:
            self.R_BITS = 12323
            self.DV = 71
            self.T1 = 134
        elif level == 0:
            self.R_BITS = 8191
            self.DV = 43
            self.T1 = 67
        elif level == -1:
            self.R_BITS = 4096
            self.DV = 32
            self.T1 = 50
        elif level == -2:
            self.R_BITS = 2048
            self.DV = 35
            self.T1 = 67
        elif level == -3:
            self.R_BITS = 1024
            self.DV = 17
            self.T1 = 33
        else:
            raise ValueError(f"Nível de segurança {level} não suportado")
        
        self.N_BITS = 2 * self.R_BITS
        self.ELL_BITS = 256
        self.ELL_SIZE = self.ELL_BITS // 8
        
        # Parâmetros do decodificador BGF otimizados
        self.TAU = max(2, self.T1 // 50)  # Threshold inicial adaptativo
        self.NB_ITER = 50  # Mais iterações para melhor convergência
        self.MIN_THRESHOLD = 1
        self.MAX_STAGNATION = 5  # Limite de estagnação
        
        # Precomputar tamanhos em bytes
        self.R_SIZE = (self.R_BITS + 7) // 8
        self.N_SIZE = (self.N_BITS + 7) // 8

In [131]:
# Instância global dos parâmetros (configurável)
PARAMS = SecurityParams(level=-3)

In [132]:
# ============================================================================
# CONFIGURAÇÃO DO ANEL POLINOMIAL
# ============================================================================
F = GF(2)
R = PolynomialRing(F, 'x')
x = R.gen()

# Polinómios modulares
mod_poly_r = x**PARAMS.R_BITS - 1
mod_poly_n = x**PARAMS.N_BITS - 1

In [133]:
# ============================================================================
# CLASSES DE DADOS
# ============================================================================
class SecretKey:
    """Chave secreta com validação"""
    def __init__(self, h0, h1, sigma):
        self.h0 = h0
        self.h1 = h1
        self.sigma = sigma
        self._validate()
    
    def _validate(self):
        """Validação básica da chave secreta"""
        if not all(hasattr(self, attr) for attr in ['h0', 'h1', 'sigma']):
            raise ValueError("Chave secreta incompleta")

class PublicKey:
    """Chave pública com validação"""
    def __init__(self, h):
        self.h = h
        if h is None:
            raise ValueError("Chave pública não pode ser None")

class Ciphertext:
    """Texto cifrado"""
    def __init__(self, c0, c1):
        self.c0 = c0
        self.c1 = c1

class SharedSecret:
    """Segredo compartilhado"""
    def __init__(self, raw):
        self.raw = raw

In [134]:
# ============================================================================
# UTILITÁRIOS CRIPTOGRÁFICOS MELHORADOS
# ============================================================================
class SecureRandom:
    """Gerador de números aleatórios criptograficamente seguro"""
    
    @staticmethod
    def secure_bytes(n_bytes: int) -> bytes:
        """Gera bytes aleatórios seguros"""
        return os.urandom(n_bytes)
    
    @staticmethod
    def secure_vector(length: int) -> vector:
        """Gera vetor aleatório seguro"""
        n_bytes = (length + 7) // 8
        rand_bytes = SecureRandom.secure_bytes(n_bytes)
        bits = []
        for byte in rand_bytes:
            for i in range(8):
                if len(bits) >= length:
                    break
                bits.append((byte >> i) & 1)
        return vector(F, bits[:length])

In [ ]:
def generate_sparse_vector_improved(length: int, weight: int, max_attempts: int = 1000) -> 'Polynomial':
    """
    Gera vetor esparso com peso exato - versão melhorada
    """
    if weight > length:
        raise ValueError(f"Peso {weight} maior que comprimento {length}")
    
    for attempt in range(max_attempts):
        try:
            positions = sorted(sample(range(length), weight))
            poly = sum(x**pos for pos in positions)
            poly = poly % (x**length - 1)
            
            # Verificação rigorosa do peso
            actual_weight = sum(1 for c in poly.list() if c != 0)
            if actual_weight == weight:
                logger.debug(f"Sparse vector gerado: peso={weight}, tentativa={attempt+1}")
                return poly
        except Exception as e:
            logger.warning(f"Tentativa {attempt+1} falhou: {e}")
    
    raise RuntimeError(f"Falha ao gerar vetor esparso após {max_attempts} tentativas")

In [136]:
def safe_polynomial_inverse(poly) -> 'Polynomial':
    """
    Inversão segura de polinômio com verificação prévia
    """
    try:
        # Verificar se é inversível antes de tentar
        gcd_result = gcd(poly, mod_poly_r)
        if gcd_result != 1:
            raise ValueError("Polinômio não é inversível (gcd != 1)")
        
        inv = poly.inverse_mod(mod_poly_r)
        
        # Verificação da inversão
        verification = (poly * inv) % mod_poly_r
        if verification != 1:
            raise ValueError("Verificação da inversão falhou")
        
        logger.debug("Inversão de polinômio bem-sucedida")
        return inv
        
    except Exception as e:
        logger.error(f"Falha na inversão: {e}")
        raise

In [137]:
# ============================================================================
# FUNÇÕES HASH MELHORADAS
# ============================================================================
class HashFunctions:
    """Funções hash otimizadas e determinísticas"""
    
    @staticmethod
    def function_H_deterministic(m_vec: vector) -> 'Polynomial':
        """
        Função H determinística e eficiente para geração do vetor de erro
        """
        m_bytes = HashFunctions._vector_to_bytes(m_vec)
        
        # Usar SHAKE-256 com contexto específico
        shake = hashlib.shake_256()
        shake.update(b"BIKE_H_FUNCTION_V2")
        shake.update(m_bytes)
        
        positions = set()
        chunk_size = 1024  # Processar em chunks para eficiência
        
        while len(positions) < PARAMS.T1:
            rand_bytes = shake.digest(chunk_size)
            
            # Processar bytes em grupos de 4 para eficiência
            for i in range(0, len(rand_bytes) - 3, 4):
                if len(positions) >= PARAMS.T1:
                    break
                
                rand_int = int.from_bytes(rand_bytes[i:i+4], 'little')
                pos = rand_int % PARAMS.N_BITS
                positions.add(pos)
        
        # Garantir exatamente T1 posições
        positions = sorted(list(positions))[:PARAMS.T1]
        
        poly = sum(x**pos for pos in positions) if positions else R(0)
        poly = poly % mod_poly_n
        
        actual_weight = sum(1 for c in poly.list() if c != 0)
        logger.debug(f"function_H: peso={actual_weight} (target: {PARAMS.T1})")
        
        return poly
    
    @staticmethod
    def function_L(e_poly: 'Polynomial') -> vector:
        """Função L para calcular hash do vetor de erro"""
        e_bytes = HashFunctions._poly_to_bytes(e_poly, PARAMS.N_BITS)
        h = hashlib.sha3_384(b"BIKE_L_FUNCTION" + e_bytes).digest()
        return HashFunctions._bytes_to_vector(h[:PARAMS.ELL_SIZE], PARAMS.ELL_BITS)
    
    @staticmethod
    def function_K(m_vec: vector, c0_poly: 'Polynomial', c1_vec: vector) -> vector:
        """Função K para derivar chave final"""
        m_bytes = HashFunctions._vector_to_bytes(m_vec)
        c0_bytes = HashFunctions._poly_to_bytes(c0_poly, PARAMS.R_BITS)
        c1_bytes = HashFunctions._vector_to_bytes(c1_vec)
        
        inp = b"BIKE_K_FUNCTION" + m_bytes + c0_bytes + c1_bytes
        h = hashlib.sha3_384(inp).digest()
        return HashFunctions._bytes_to_vector(h[:PARAMS.ELL_SIZE], PARAMS.ELL_BITS)
    
    @staticmethod
    def _vector_to_bytes(vec: vector) -> bytes:
        """Converte vetor para bytes de forma eficiente"""
        data = bytearray((len(vec) + 7) // 8)
        for i, bit in enumerate(vec):
            if int(bit) == 1:
                data[i // 8] |= (1 << (i % 8))
        return bytes(data)
    
    @staticmethod
    def _bytes_to_vector(data: bytes, target_bits: int) -> vector:
        """Converte bytes para vetor de bits"""
        bits = []
        for byte in data:
            for i in range(8):
                if len(bits) >= target_bits:
                    break
                bits.append((byte >> i) & 1)
        while len(bits) < target_bits:
            bits.append(0)
        return vector(F, bits[:target_bits])
    
    @staticmethod
    def _poly_to_bytes(poly: 'Polynomial', target_bits: int) -> bytes:
        """Converte polinômio para bytes"""
        coeffs = poly.list()
        if len(coeffs) < target_bits:
            coeffs.extend([0] * (target_bits - len(coeffs)))
        vec = vector(F, coeffs[:target_bits])
        return HashFunctions._vector_to_bytes(vec)

In [138]:
def create_circulant_matrix(poly, r_bits):
    """
    Cria matriz circulante a partir de polinômio.
    Função auxiliar caso não esteja disponível no objeto de parâmetros.
    """
    from sage.all import Matrix, GF
    
    F = GF(2)
    coeffs = poly.list()
    if len(coeffs) < r_bits:
        coeffs.extend([0] * (r_bits - len(coeffs)))
    
    circulant = Matrix(F, r_bits, r_bits)
    
    for i in range(r_bits):
        for j in range(r_bits):
            idx = (j - i) % r_bits
            circulant[i, j] = F(coeffs[idx])
    
    return circulant

In [139]:
class ImprovedBGFDecoder:
    """
    Decodificador BGF melhorado com múltiplas estratégias para escapar de mínimos locais
    """
    
    def __init__(self):
        self._circulant_cache = {}
        self.debug = True
        self.MAX_STAGNATION = 10  # Adicionar isto!

    def _compute_dynamic_threshold(self, syndrome_weight: int) -> int:
        """Calcula threshold dinâmico baseado no peso da síndrome"""
        # Fórmula da implementação NIST
        threshold = (syndrome_weight * (PARAMS.DV - 1)) // (2 * PARAMS.R_BITS)
        return max(1, threshold) 
    
    def _create_circulant_matrix_cached(self, poly: 'Polynomial') -> 'Matrix':
        """Cria matriz circulante com cache para eficiência"""
        poly_key = tuple(poly.list())
        
        if poly_key in self._circulant_cache:
            return self._circulant_cache[poly_key]
        
        coeffs = poly.list()
        if len(coeffs) < PARAMS.R_BITS:
            coeffs.extend([0] * (PARAMS.R_BITS - len(coeffs)))
        
        circulant = Matrix(F, PARAMS.R_BITS, PARAMS.R_BITS)
        
        for i in range(PARAMS.R_BITS):
            for j in range(PARAMS.R_BITS):
                idx = (j - i) % PARAMS.R_BITS
                circulant[i, j] = F(coeffs[idx])
        
        self._circulant_cache[poly_key] = circulant
        return circulant
    
    def decode(self, syndrome_vec: vector, h0_poly: 'Polynomial', h1_poly: 'Polynomial') -> 'Polynomial':
        """
        Decodificação BGF com múltiplas estratégias anti-mínimos locais
        """
        start_time = time.time()
        logger.info("Iniciando decodificação BGF aprimorada")
        
        # Criar matrizes de paridade
        H0 = self._create_circulant_matrix_cached(h0_poly)
        H1 = self._create_circulant_matrix_cached(h1_poly)
        H = block_matrix([[H0, H1]], subdivide=False)
        
        s_target = vector(F, [F(int(x)) for x in syndrome_vec])
        
        # Múltiplas tentativas com diferentes inicializações
        best_result = None
        best_distance = float('inf')
        
        strategies = [
            ('zero_init', self._decode_with_zero_init),
            ('random_init', self._decode_with_random_init),
            ('smart_init', self._decode_with_smart_init),
            ('hybrid_search', self._decode_with_hybrid_search)
        ]
        
        for strategy_name, strategy_func in strategies:
            logger.info(f"Tentando estratégia: {strategy_name}")
            
            try:
                result, distance = strategy_func(H, s_target, h0_poly, h1_poly)
                
                if distance == 0:  # Solução perfeita encontrada
                    logger.info(f"Solução perfeita encontrada com {strategy_name}")
                    return result
                
                if distance < best_distance:
                    best_distance = distance
                    best_result = result
                    logger.info(f"{strategy_name}: nova melhor distância = {distance}")
                    
            except Exception as e:
                logger.warning(f"Estratégia {strategy_name} falhou: {e}")
                continue
        
        # Se chegamos aqui, usar a melhor solução encontrada
        if best_result is not None:
            elapsed = time.time() - start_time
            logger.info(f"BGF concluído: melhor distância={best_distance}, tempo={elapsed:.3f}s")
            return best_result
        
        # Último recurso: gerar erro sintético
        logger.warning("Todas as estratégias falharam, gerando erro sintético")
        return self._generate_synthetic_error()
    
    def _decode_with_zero_init(self, H: 'Matrix', s_target: vector, h0_poly, h1_poly) -> Tuple['Polynomial', int]:
        """Estratégia 1: Inicialização zero com threshold adaptativo"""
        e = vector(F, PARAMS.N_BITS)
        return self._bgf_core_adaptive(H, e, s_target)
    
    def _decode_with_random_init(self, H: 'Matrix', s_target: vector, h0_poly, h1_poly) -> Tuple['Polynomial', int]:
        """Estratégia 2: Inicialização aleatória"""
        # Inicialização com peso baixo aleatório
        initial_weight = min(PARAMS.T1 // 4, 20)
        positions = sample(range(PARAMS.N_BITS), initial_weight)
        
        e = vector(F, PARAMS.N_BITS)
        for pos in positions:
            e[pos] = F(1)
        
        return self._bgf_core_adaptive(H, e, s_target)
    
    def _decode_with_smart_init(self, H: 'Matrix', s_target: vector, h0_poly, h1_poly) -> Tuple['Polynomial', int]:
        """Estratégia 3: Inicialização inteligente baseada na síndrome"""
        e = vector(F, PARAMS.N_BITS)
        
        # Análise da síndrome para identificar posições prováveis
        syndrome_analysis = self._analyze_syndrome(H, s_target)
        
        # Inicializar com as posições mais prováveis
        for pos in syndrome_analysis[:min(len(syndrome_analysis), PARAMS.T1 // 3)]:
            e[pos] = F(1)
        
        return self._bgf_core_adaptive(H, e, s_target)
    
    def _decode_with_hybrid_search(self, H: 'Matrix', s_target: vector, h0_poly, h1_poly) -> Tuple['Polynomial', int]:
        """Estratégia 4: Busca híbrida com múltiplas tentativas"""
        best_e = None
        best_distance = float('inf')
        
        # Múltiplas tentativas com perturbações
        for attempt in range(3):
            # Inicialização baseada no attempt
            if attempt == 0:
                e = vector(F, PARAMS.N_BITS)  # Zero
            elif attempt == 1:
                e = SecureRandom.secure_vector(PARAMS.N_BITS)  # Aleatório completo
            else:
                # Híbrido: baseado na melhor solução anterior com perturbação
                e = vector(F, best_e) if best_e else vector(F, PARAMS.N_BITS)
                self._perturb_vector(e, PARAMS.T1 // 10)
            
            try:
                result_e, distance = self._bgf_core_enhanced(H, e, s_target)
                
                if distance < best_distance:
                    best_distance = distance
                    best_e = result_e
                    
                if distance == 0:
                    break
                    
            except Exception as e_exc:
                logger.warning(f"Tentativa híbrida {attempt} falhou: {e_exc}")
                continue
        
        if best_e is None:
            raise RuntimeError("Busca híbrida falhou completamente")
        
        return self._vector_to_polynomial(best_e), best_distance
    
    def _bgf_core_adaptive(self, H: 'Matrix', e: vector, s_target: vector) -> Tuple['Polynomial', int]:
        """Core BGF com threshold adaptativo"""
        max_iterations = 150

        # Histórico para detecção de estagnação
        best_e = vector(F, e)
        best_distance = float('inf')
        stagnation_count = 0

        for iteration in range(max_iterations):
            s_current = H * e
            current_distance = sum(1 for i in range(len(s_current)) if s_current[i] != s_target[i])

            if current_distance == 0:
                return self._vector_to_polynomial(e), 0

            # Atualizar melhor solução
            if current_distance < best_distance:
                best_distance = current_distance
                best_e = vector(F, e)
                stagnation_count = 0
            else:
                stagnation_count += 1

            # Calcular threshold dinamicamente
            syndrome_weight = current_distance
            threshold = self._compute_dynamic_threshold(syndrome_weight)

            # Log periódico
            if iteration % 10 == 0:
                actual_weight = sum(1 for i in range(len(e)) if e[i] == F(1))
                logger.debug(f"Iter {iteration}: dist={current_distance}, threshold={threshold}, e_weight={actual_weight}")

            # Detectar e escapar de mínimo local
            if stagnation_count >= self.MAX_STAGNATION:
                if self._escape_local_minimum(e, H, s_target, current_distance):
                    stagnation_count = 0
                    continue
                
            # BGF padrão
            unsatisfied_count = self._compute_unsatisfied_counts(H, s_current, s_target)
            bits_to_flip = self._select_bits_adaptive(unsatisfied_count, threshold, iteration)

            # Aplicar flips
            for j in bits_to_flip:
                e[j] = F(1) - e[j]

        return self._vector_to_polynomial(best_e), best_distance

    def _bgf_core_enhanced(self, H: 'Matrix', e: vector, s_target: vector) -> Tuple[vector, int]:
        """Versão aprimorada do core BGF com mais estratégias"""
        max_iterations = 100
        
        # Parâmetros adaptativos
        threshold = max(2, PARAMS.T1 // 35)
        momentum_factor = 0.7
        previous_flips = set()
        
        best_e = vector(F, e)
        best_distance = float('inf')
        
        for iteration in range(max_iterations):
            s_current = H * e
            current_distance = sum(1 for i in range(len(s_current)) if s_current[i] != s_target[i])
            
            if current_distance == 0:
                return e, 0
            
            if current_distance < best_distance:
                best_distance = current_distance
                best_e = vector(F, e)
            
            # Cálculo com momentum
            unsatisfied_count = self._compute_unsatisfied_counts(H, s_current, s_target)
            bits_to_flip = self._select_bits_with_momentum(
                unsatisfied_count, threshold, previous_flips, momentum_factor
            )
            
            # Aplicar flips
            current_flips = set()
            for j in bits_to_flip:
                e[j] = F(1) - e[j]
                current_flips.add(j)
            
            previous_flips = current_flips
            
            # Ajuste dinâmico
            if iteration % 10 == 0 and current_distance >= best_distance:
                threshold = max(1, threshold - 1)
        
        return best_e, best_distance
    
    def _escape_local_minimum(self, e: vector, H: 'Matrix', s_target: vector, current_distance: int) -> bool:
        """Múltiplas estratégias para escapar de mínimos locais"""
        original_e =  vector(F, e)
        
        strategies = [
            ('random_perturbation', self._random_perturbation),
            ('guided_perturbation', self._guided_perturbation),
            ('high_unsatisfied_flip', self._high_unsatisfied_flip),
            ('pattern_breaking', self._pattern_breaking)
        ]
        
        for strategy_name, strategy_func in strategies:
            e_test =  vector(F, original_e)
            
            try:
                strategy_func(e_test, H, s_target)
                
                # Testar se a perturbação ajudou
                s_test = H * e_test
                test_distance = sum(1 for i in range(len(s_test)) if s_test[i] != s_target[i])
                
                if test_distance < current_distance:
                    # Aplicar a perturbação bem-sucedida
                    for i in range(len(e)):
                        e[i] = e_test[i]
                    logger.debug(f"Escape bem-sucedido com {strategy_name}")
                    return True
                    
            except Exception as exc:
                logger.debug(f"Estratégia de escape {strategy_name} falhou: {exc}")
                continue
        
        return False
    
    def _random_perturbation(self, e: vector, H: 'Matrix', s_target: vector):
        """Perturbação aleatória controlada"""
        perturbation_size = min(20, PARAMS.T1 // 10)
        positions = sample(range(PARAMS.N_BITS), perturbation_size)
        
        for pos in positions:
            e[pos] = F(1) - e[pos]
    
    def _guided_perturbation(self, e: vector, H: 'Matrix', s_target: vector):
        """Perturbação guiada pela análise da síndrome"""
        s_current = H * e
        unsatisfied_count = self._compute_unsatisfied_counts(H, s_current, s_target)
        
        # Focar em bits com contagem média-alta
        mean_unsatisfied = sum(unsatisfied_count) / len(unsatisfied_count)
        candidates = [i for i, count in enumerate(unsatisfied_count) 
                     if count >= mean_unsatisfied * 0.7]
        
        if candidates:
            flip_count = min(15, len(candidates) // 2)
            positions = sample(candidates, flip_count)
            
            for pos in positions:
                e[pos] = F(1) - e[pos]
    
    def _high_unsatisfied_flip(self, e: vector, H: 'Matrix', s_target: vector):
        """Inverter bits com maior contagem de insatisfação"""
        s_current = H * e
        unsatisfied_count = self._compute_unsatisfied_counts(H, s_current, s_target)
        
        # Pegar os 10% com maior contagem
        sorted_indices = sorted(range(len(unsatisfied_count)), 
                               key=lambda i: unsatisfied_count[i], reverse=True)
        
        flip_count = max(5, len(sorted_indices) // 10)
        for i in range(flip_count):
            pos = sorted_indices[i]
            e[pos] = F(1) - e[pos]
    
    def _pattern_breaking(self, e: vector, H: 'Matrix', s_target: vector):
        """Quebrar padrões detectados no vetor de erro"""
        # Detectar clusters de 1s e quebrar alguns
        ones_positions = [i for i in range(len(e)) if e[i] == F(1)]
        
        if len(ones_positions) > 10:
            # Remover alguns 1s aleatoriamente
            to_remove = sample(ones_positions, min(5, len(ones_positions) // 4))
            for pos in to_remove:
                e[pos] = F(0)
            
            # Adicionar alguns 1s em posições diferentes
            zeros_positions = [i for i in range(len(e)) if e[i] == F(0)]
            if zeros_positions:
                to_add = sample(zeros_positions, min(5, len(zeros_positions) // 20))
                for pos in to_add:
                    e[pos] = F(1)
    
    def _analyze_syndrome(self, H: 'Matrix', s_target: vector) -> List[int]:
        """Análise da síndrome para identificar posições prováveis de erro"""
        candidates = []
        
        for j in range(PARAMS.N_BITS):
            score = 0
            for i in range(len(s_target)):
                if s_target[i] == F(1) and H[i, j] == F(1):
                    score += 1
            
            if score > 0:
                candidates.append((j, score))
        
        # Ordenar por score e retornar posições
        candidates.sort(key=lambda x: x[1], reverse=True)
        return [pos for pos, score in candidates]
    
    def _select_bits_adaptive(self, unsatisfied_count: List[int], threshold: int, iteration: int) -> List[int]:
        """Seleção de bits mais próxima da implementação C"""
        if not unsatisfied_count:
            return []
        
        # Selecionar TODOS os bits acima do threshold
        bits_to_flip = [j for j in range(len(unsatisfied_count)) 
                       if unsatisfied_count[j] >= threshold]
        
        # Se nenhum bit selecionado e threshold > 1, reduzir threshold
        if not bits_to_flip and threshold > 1:
            bits_to_flip = [j for j in range(len(unsatisfied_count)) 
                           if unsatisfied_count[j] >= (threshold - 1)]
        
        return bits_to_flip
    
    def _select_bits_with_momentum(self, unsatisfied_count: List[int], threshold: int, 
                                  previous_flips: set, momentum_factor: float) -> List[int]:
        """Seleção de bits com momentum das iterações anteriores"""
        base_bits = self._select_bits_adaptive(unsatisfied_count, threshold, 0)
        
        # Aplicar momentum: preferir bits que foram flipped recentemente
        momentum_bits = []
        for bit in previous_flips:
            if random.random() < momentum_factor:
                momentum_bits.append(bit)
        
        # Combinar e remover duplicatas
        combined_bits = list(set(base_bits + momentum_bits))
        
        return combined_bits
    
    def _adjust_threshold(self, current_threshold: int, distance: int, iteration: int) -> int:
        """Ajuste adaptativo do threshold baseado no progresso"""
        if distance > PARAMS.R_BITS // 4:  # Distância muito alta
            return max(1, current_threshold - 1)
        elif distance < PARAMS.R_BITS // 20:  # Distância baixa
            return min(current_threshold + 1, PARAMS.T1 // 10)
        else:
            return current_threshold
    
    def _compute_unsatisfied_counts(self, H: 'Matrix', s_current: vector, s_target: vector) -> List[int]:
        """Computa contadores de insatisfação otimizado"""
        unsatisfied_count = [0] * PARAMS.N_BITS
        
        for i in range(len(s_current)):
            if s_current[i] != s_target[i]:
                for j in range(PARAMS.N_BITS):
                    if H[i, j] == F(1):
                        unsatisfied_count[j] += 1
        
        return unsatisfied_count
    
    def _perturb_vector(self, e: vector, perturbation_size: int):
        """Aplica perturbação controlada ao vetor"""
        positions = sample(range(len(e)), min(perturbation_size, len(e)))
        for pos in positions:
            e[pos] = F(1) - e[pos]
    
    def _vector_to_polynomial(self, e: vector) -> 'Polynomial':
        """Converte vetor para polinômio"""
        positions = [i for i in range(len(e)) if e[i] == F(1)]
        
        if not positions:
            return R(0)
        
        poly = sum(x**pos for pos in positions)
        return poly % mod_poly_n
    
    def _generate_synthetic_error(self) -> 'Polynomial':
        """Gera erro sintético como último recurso"""
        logger.warning("Gerando vetor de erro sintético")
        positions = sorted(sample(range(PARAMS.N_BITS), min(PARAMS.T1, PARAMS.N_BITS)))
        return sum(x**pos for pos in positions) % mod_poly_n

In [140]:

# Instância global do decodificador
improved_bgf_decoder = ImprovedBGFDecoder()

In [141]:
# ============================================================================
# FUNÇÕES PRINCIPAIS DO BIKE
# ============================================================================
def generate_keypair() -> Tuple[PublicKey, SecretKey]:
    """Gera par de chaves com validação rigorosa"""
    logger.info("Gerando par de chaves BIKE")
    
    max_attempts = 100
    for attempt in range(max_attempts):
        try:
            # Gerar vetores esparsos
            h0 = generate_sparse_vector_improved(PARAMS.R_BITS, PARAMS.DV)
            h1 = generate_sparse_vector_improved(PARAMS.R_BITS, PARAMS.DV)
            
            # Gerar sigma com aleatoriedade segura
            sigma = SecureRandom.secure_vector(PARAMS.ELL_BITS)
            
            # Calcular chave pública
            h0_inv = safe_polynomial_inverse(h0)
            h = (h1 * h0_inv) % mod_poly_r
            
            sk = SecretKey(h0, h1, sigma)
            pk = PublicKey(h)
            
            logger.info(f"Par de chaves gerado com sucesso (tentativa {attempt + 1})")
            return pk, sk
            
        except Exception as e:
            logger.warning(f"Tentativa {attempt + 1} falhou: {e}")
            if attempt == max_attempts - 1:
                raise RuntimeError("Falha ao gerar par de chaves após múltiplas tentativas")

In [142]:
# ============================================================================
# FUNÇÕES AUXILIARES OTIMIZADAS
# ============================================================================
def split_vector_improved(e_poly: 'Polynomial') -> Tuple['Polynomial', 'Polynomial']:
    """Divisão otimizada do vetor de erro"""
    coeffs = e_poly.list()
    if len(coeffs) < PARAMS.N_BITS:
        coeffs.extend([0] * (PARAMS.N_BITS - len(coeffs)))
    
    # e0: primeiros R_BITS coeficientes
    e0_coeffs = coeffs[:PARAMS.R_BITS]
    e0 = sum(F(e0_coeffs[i]) * x**i for i in range(len(e0_coeffs)) if e0_coeffs[i] != 0)
    e0 = e0 % mod_poly_r
    
    # e1: próximos R_BITS coeficientes  
    e1_coeffs = coeffs[PARAMS.R_BITS:PARAMS.N_BITS]
    e1 = sum(F(e1_coeffs[i]) * x**i for i in range(len(e1_coeffs)) if e1_coeffs[i] != 0)
    e1 = e1 % mod_poly_r
    
    return e0, e1

In [143]:
def xor_vectors_improved(a: vector, b: vector) -> vector:
    """XOR otimizado entre vetores"""
    max_len = max(len(a), len(b))
    a_list = list(a) + [F(0)] * (max_len - len(a))
    b_list = list(b) + [F(0)] * (max_len - len(b))
    return vector(F, [a_list[i] + b_list[i] for i in range(max_len)])

In [144]:
def compute_syndrome_optimized(c0_poly: 'Polynomial', h0_poly: 'Polynomial') -> vector:
    """Cálculo otimizado da síndrome"""
    prod = (c0_poly * h0_poly) % mod_poly_r
    
    coeffs = prod.list()
    if len(coeffs) < PARAMS.R_BITS:
        coeffs.extend([0] * (PARAMS.R_BITS - len(coeffs)))
    
    return vector(F, [F(int(c)) for c in coeffs[:PARAMS.R_BITS]])

In [145]:
def verify_consistency(m_prime: vector, ct: Ciphertext, sk: SecretKey) -> bool:
    """Verificação rigorosa de consistência"""
    try:
        # Recomputar vetor de erro
        e_check = HashFunctions.function_H_deterministic(m_prime)
        e0_check, e1_check = split_vector_improved(e_check)
        
        # Recomputar c0
        h0_inv = safe_polynomial_inverse(sk.h0)
        h_reconstructed = (sk.h1 * h0_inv) % mod_poly_r
        temp_check = (e1_check * h_reconstructed) % mod_poly_r
        c0_check = (e0_check + temp_check) % mod_poly_r
        
        # Verificação em tempo constante (aproximado)
        diff = (ct.c0 - c0_check) % mod_poly_r
        return diff == 0
        
    except Exception as e:
        logger.error(f"Erro na verificação de consistência: {e}")
        return False

In [146]:
def encaps(pk: PublicKey) -> Tuple[Ciphertext, SharedSecret]:
    """Encapsulamento com validação"""
    logger.info("Iniciando encapsulamento")
    
    # Gerar mensagem aleatória segura
    m = SecureRandom.secure_vector(PARAMS.ELL_BITS)
    
    # Gerar vetor de erro
    e_poly = HashFunctions.function_H_deterministic(m)
    e0_poly, e1_poly = split_vector_improved(e_poly)
    
    # Calcular c0 com operações no anel correto
    temp = (e1_poly * pk.h) % mod_poly_r
    c0 = (e0_poly + temp) % mod_poly_r
    
    # Calcular c1
    L_vec = HashFunctions.function_L(e_poly)
    c1 = xor_vectors_improved(L_vec, m)
    
    # Derivar segredo compartilhado
    ss = HashFunctions.function_K(m, c0, c1)
    
    logger.info("Encapsulamento concluído com sucesso")
    return Ciphertext(c0, c1), SharedSecret(ss)


In [147]:
def decaps_improved(ct: Ciphertext, sk: SecretKey, pk: PublicKey) -> SharedSecret:
    """
    Decapsulamento melhorado com decodificador anti-mínimos locais
    """
    logger.info("Iniciando decapsulamento com decodificador melhorado")
    
    try:
        # Calcular síndrome
        syndrome = compute_syndrome_optimized(ct.c0, sk.h0)
        logger.debug(f"Síndrome calculada: peso = {sum(1 for x in syndrome if x != 0)}")
        
        # Usar o decodificador melhorado
        e_prime_poly = improved_bgf_decoder.decode(syndrome, sk.h0, sk.h1)
        
        # Verificar o peso do erro decodificado
        e_weight = sum(1 for c in e_prime_poly.list() if c != 0)
        logger.info(f"Erro decodificado com peso: {e_weight}")
        
        # Calcular L' e recuperar mensagem
        L_prime = HashFunctions.function_L(e_prime_poly)
        m_prime = xor_vectors_improved(ct.c1, L_prime)
        
        # Verificação de consistência rigorosa
        logger.debug("Iniciando verificação de consistência...")
        
        # Recomputar vetor de erro esperado
        e_check = HashFunctions.function_H_deterministic(m_prime)
        e0_check, e1_check = split_vector_improved(e_check)
        
        # Recomputar c0 esperado
        temp_check = (e1_check * pk.h) % mod_poly_r
        c0_check = (e0_check + temp_check) % mod_poly_r
        
        # Verificar consistência
        is_consistent = ((ct.c0 - c0_check) % mod_poly_r == 0)
        
        if is_consistent:
            logger.info("✅ Decapsulamento consistente - usando mensagem recuperada")
            k = HashFunctions.function_K(m_prime, ct.c0, ct.c1)
        else:
            logger.warning("⚠️ Inconsistência detectada - usando fallback sigma")
            
            # Log detalhado para debug
            logger.debug(f"c0 original: {ct.c0}")
            logger.debug(f"c0 recomputado: {c0_check}")
            logger.debug(f"Diferença: {(ct.c0 - c0_check) % mod_poly_r}")
            
            # Usar sigma como fallback
            k = HashFunctions.function_K(sk.sigma, ct.c0, ct.c1)
        
        logger.info("Decapsulamento concluído")
        return SharedSecret(k)
        
    except Exception as e:
        logger.error(f"Erro crítico no decapsulamento: {e}")
        logger.exception("Stack trace completo:")
        
        # Fallback de emergência
        logger.warning("Usando fallback de emergência com sigma")
        k_emergency = HashFunctions.function_K(sk.sigma, ct.c0, ct.c1)
        return SharedSecret(k_emergency)

In [148]:

"""
Teste da implementação BIKE melhorada
"""
print("=" * 60)
print("TESTE BIKE COM DECODIFICADOR MELHORADO")
print("=" * 60)

success_count = 0
total_tests = 5

for test_num in range(total_tests):
    print(f"\n🧪 Teste {test_num + 1}/{total_tests}")
    
    try:
        start_total = time.time()
        
        # Geração de chaves
        print("🔑 Gerando chaves...")
        start_time = time.time()
        pk, sk = generate_keypair()
        keygen_time = time.time() - start_time
        print(f"   ✅ Chaves geradas em {keygen_time:.3f}s")
        
        # Encapsulamento
        print("📦 Encapsulando...")
        start_time = time.time()
        ct, ss_enc = encaps(pk)
        encaps_time = time.time() - start_time
        print(f"   ✅ Encapsulamento em {encaps_time:.3f}s")
        
        # Decapsulamento melhorado
        print("🔓 Decapsulando (versão melhorada)...")
        start_time = time.time()
        ss_dec = decaps_improved(ct, sk, pk)
        decaps_time = time.time() - start_time
        print(f"   ✅ Decapsulamento em {decaps_time:.3f}s")
        
        # Verificação
        secrets_match = list(ss_enc.raw) == list(ss_dec.raw)
        total_time = time.time() - start_total
        
        print(f"   ⏱️ Tempo total: {total_time:.3f}s")
        
        if secrets_match:
            print("   ✅ SUCESSO: Segredos coincidem!")
            success_count += 1
        else:
            print("   ❌ FALHA: Segredos não coincidem")
            print(f"   Enc: {ss_enc.raw[:10]}...")
            print(f"   Dec: {ss_dec.raw[:10]}...")
            
    except Exception as e:
        print(f"   ❌ ERRO: {e}")
        logger.exception(f"Erro no teste {test_num + 1}")

if success_count == total_tests:
    print("🎉 TODOS OS TESTES PASSARAM!")
elif success_count > 0:
    print("⚠️ SUCESSO PARCIAL - Verifique os logs para detalhes")
else:
    print("❌ TODOS OS TESTES FALHARAM - Revise a implementação")

TESTE BIKE COM DECODIFICADOR MELHORADO

🧪 Teste 1/5
🔑 Gerando chaves...
   ✅ Chaves geradas em 0.005s
📦 Encapsulando...
   ✅ Encapsulamento em 0.017s
🔓 Decapsulando (versão melhorada)...
   ✅ Decapsulamento em 140.197s
   ⏱️ Tempo total: 140.220s
   ❌ FALHA: Segredos não coincidem
   Enc: (0, 1, 1, 0, 1, 0, 0, 1, 1, 0)...
   Dec: (1, 1, 1, 0, 0, 0, 1, 1, 0, 1)...

🧪 Teste 2/5
🔑 Gerando chaves...
   ✅ Chaves geradas em 0.003s
📦 Encapsulando...
   ✅ Encapsulamento em 0.011s
🔓 Decapsulando (versão melhorada)...
   ✅ Decapsulamento em 134.935s
   ⏱️ Tempo total: 134.950s
   ❌ FALHA: Segredos não coincidem
   Enc: (1, 1, 1, 1, 0, 1, 1, 1, 0, 1)...
   Dec: (0, 1, 0, 0, 0, 1, 0, 0, 0, 1)...

🧪 Teste 3/5
🔑 Gerando chaves...
   ✅ Chaves geradas em 0.003s
📦 Encapsulando...
   ✅ Encapsulamento em 0.011s
🔓 Decapsulando (versão melhorada)...
   ✅ Decapsulamento em 133.726s
   ⏱️ Tempo total: 133.740s
   ❌ FALHA: Segredos não coincidem
   Enc: (0, 1, 0, 0, 0, 0, 1, 0, 1, 1)...
   Dec: (0, 0, 1, 1, 0